<a href="https://colab.research.google.com/github/maskot1977/AdvancedTheoryOfPharmacoinformaticsSimulation/blob/main/%E3%83%95%E3%82%A1%E3%83%BC%E3%83%9E%E3%82%B3%E3%82%A4%E3%83%B3%E3%83%95%E3%82%A9%E3%83%9E%E3%83%86%E3%82%A3%E3%82%AF%E3%82%B9%E3%82%B7%E3%83%9F%E3%83%A5%E3%83%AC%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3%E7%89%B9%E8%AB%961.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ファーマコインフォマティクスシミュレーション特論

　　　小寺 正明 

# 第1回：過剰適合（過学習）

過剰適合とは、ある特定の問題に過剰に適合した結果、汎化性能を失うことをさします。「過学習」という言葉の方が知名度が高いですが、誤解を招きやすい言葉だと個人的には思っていて、「過剰適合」という言葉の方が私は好きです。

過剰適合の様子をPythonコードで実演してみましょう。まず、次のような関数を用意します。

In [ ]:
# 関数 f(x) を定義します。
import numpy as np

f = lambda x: 0.05*x+0.8*np.sin(3/4 * x) # 講義中はこの関数を用います。
#f = lambda x: 0.2*np.sin(x) + 0.3*np.cos(2*x) + 0.5*np.sin(2/3*x) + 0.2*np.sin(x/3) # 課題で用います。

In [ ]:
# x の定義域を決めます。
x_test = np.linspace(-10, 10, 101) # test data
x_test

In [ ]:
# 真の f(x) の値
t_test = f(x_test)
t_test

In [ ]:
# 描画します。
import matplotlib.pyplot as plt

plt.plot(x_test, t_test, label="true f(x) (test data)")
plt.grid()
plt.legend()
plt.xlabel("x")
plt.ylabel("t = f(x)")
plt.show()

何か注目している現象があるとして、その現象の「真の姿」は上記の関数に従うが、私たちはその関数を知らないものとします。何度か実験を行うことにより、いくつかの x に対してその t = f(x) の値は測定することができます。しかし、無限回数の実験を行うことはできませんし、測定値には常に誤差が伴います。そこで、<b>限られた回数の誤差を含む測定実験から、真の姿をどのくらい正しく予測できるか</b>をシミュレーションしてみたいと思います。

機械学習では、データを次のように分割することがあります（用いる用語は、人によって異なる場合があります）。

- **training data（訓練データ、教師データ、学習データ）**
    - 機械学習モデルを訓練するために用いるデータ。説明変数と目的変数の組から成る。
- **validation data（検証データ）**
    - 機械学習モデルの性能を評価するために用いるデータ。説明変数と目的変数の組から成る。
- **test data（テストデータ）**
    - 学習に用いず、実際に予測値を出したいデータ。目的変数は必ずしも明らかになっていない。

ここでは、訓練データ、検証データ、テストデータの説明変数 $x$ を次のように決めます。

In [ ]:
x_train = np.linspace(-10, 10, 11) # training data
x_valid = np.linspace(-9, 9, 10) # validation data




ここで、訓練データや検証データに対応する目的変数 $y$ の値には、何らかの誤差が含まれているものとします。図示してみましょう。

In [ ]:
epsilon = 0.1 # 誤差の大きさを決める定数
t_train = f(x_train) + epsilon * np.cos(1.9*x_train + 1.3) # 誤差の含まれた教師データ
t_valid = f(x_valid) + epsilon * np.cos(1.7*x_valid + 1.1) # 誤差の含まれた検証データ

In [ ]:
plt.scatter(x_train, t_train, marker='o', label="training data")
plt.scatter(x_valid, t_valid, marker='x', label="validation data")
plt.plot(x_test, t_test, label="true f(x) (test data)")
plt.grid()
plt.legend()
plt.xlabel("x")
plt.ylabel("y")
plt.show()

ここで問題は、**訓練データだけから、検証データやテストデータを正しく予測できるか？** ということになります。

機械学習手法はたくさんありますが、ここでは 

- Support Vector Machine (SVM)
- K-Nearest Neighbors (KNN)
- Random Forest
- Gradient Boosting
- Multi-layer Perceptron (MLP) 

を取扱います。

## SVM (SVR)

どの機械学習モデルでも、「モデルの作成」「学習」「予測」「性能評価」という流れになります。

ここでは、SVMがどのように予測問題を解くかの説明は省略します。使い方を理解することと、SVMによる予測がどのような「クセ」を持っているのかというイメージをつかんでいただけたらと思います。

In [ ]:
# 機械学習モデルを作成する
from sklearn.svm import SVR

params = {"kernel":"rbf", "C":1e64, "gamma":1e8}
model = SVR(**params)

In [ ]:
# 学習する
model.fit(x_train.reshape(-1, 1), t_train)

In [ ]:
# 訓練データを予測
y_train = model.predict(x_train.reshape(-1, 1))
y_train

In [ ]:
# 検証データを予測
y_valid = model.predict(x_valid.reshape(-1, 1))
y_valid

In [ ]:
# テストデータを予測
y_test = model.predict(x_test.reshape(-1, 1))
y_test

In [ ]:
# 訓練データの予測性能の評価（平均絶対誤差）
from sklearn.metrics import mean_absolute_error

mean_absolute_error(t_train, y_train)

In [ ]:
# 検証データの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_valid, y_valid)

In [ ]:
# テストデータの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_test, y_test)

以上の計算では、性能評価指標として「平均絶対誤差」を使いました（性能評価指標は他にもあります）。この数字が小さいほど、機械学習モデルの性能が優れているということになります。

それでは、訓練データ、検証データ、テストデータで、どの指標が最も良いでしょうか？

普通は、訓練データの指標が最も良くなります。訓練データを用いて訓練しているので、当たり前ですね。検証データやテストデータの指標が、たまたま、最も良くなることはありえますが、普通は訓練データの指標よりも悪い数字が出ます。

数字だけでは、どのような予測結果になったのかイメージしにくいので、予測結果を図示してみましょう。

In [ ]:
plt.scatter(x_train, t_train, marker='o', label="train")
plt.scatter(x_valid, t_valid, marker='x', label="valid")
plt.plot(x_test, t_test, label="test (true)")
plt.plot(x_test, y_test, label="test (predicted)")
plt.legend()
plt.show()

これは良い予測結果と思えるでしょうか？

思えませんよね。訓練データの指標が良くなるように無理矢理合わせているだけで、検証データやテストデータの予測性能は非常に悪いことが分かると思います。これが overfitting の例です。

そして、無理矢理合わせるときに、こんな形をして合わせようとするのが SVM の「クセ」です。

今回は、わざと、SVMのハイパーパラメーターとして無茶苦茶な値を設定しました。後に、ハイパーパラメーターを適切な値に設定する（チューニングする）方法を解説します。

# Random Forest

ハイパーパラメーターのチューニングは後回しにして、今度は、SVM以外の他の機械学習モデルを使ってみることにしましょう。

Random Forest を使ってみます。ここでも、Random Forest がどんな原理で動いているかの説明は省略します。使い方を理解して、Random Forest がどんな「クセ」を持っているのかイメージをつかんでください。

In [ ]:
# 機械学習モデルを作成する
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

In [ ]:
# 学習する
model.fit(x_train.reshape(-1, 1), t_train)

In [ ]:
# 訓練データを予測
y_train = model.predict(x_train.reshape(-1, 1))
y_train

In [ ]:
# 検証データを予測
y_valid = model.predict(x_valid.reshape(-1, 1))
y_valid

In [ ]:
# テストデータを予測
y_test = model.predict(x_test.reshape(-1, 1))
y_test

In [ ]:
# 訓練データの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_train, y_train)

In [ ]:
# 検証データの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_valid, y_valid)

In [ ]:
# テストデータの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_test, y_test)

処理の流れは先程の SVM のときとほぼ一緒だということがお分かりいただけたでしょうか。違うことは、ハイパーパラメーターとして、SVMのときは（わざと）無茶苦茶なパラメータを設定した一方で、今回の Random Forest ではデフォルトの値（何も指定しなかったときに自動的に指定される値）を用いたことくらいです。

今回もまた、訓練データの指標、検証データの指標、テストデータの指標を比べてみましょう。前回と違って、ほぼ変わらない値になったのではないでしょうか。

どのような予測結果になったか図示してみましょう。

In [ ]:
plt.scatter(x_train, t_train, marker='o', label="train")
plt.scatter(x_valid, t_valid, marker='x', label="valid")
plt.plot(x_test, t_test, label="test (true)")
plt.plot(x_test, y_test, label="test (predicted)")
plt.legend()
plt.show()

さて、この結果は良い結果と言えるでしょうか？ overfitting は回避できているかもしれませんが、予測性能が高いかと言うと、そんなことはなさそうですね。

そして、カクカクした不連続な曲線で間を取ろうとするのが Random Forest の「クセ」です。

# K Neighbors

同様にして、K Neighbors も試してみましょう。

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor()

In [ ]:
# 学習する
model.fit(x_train.reshape(-1, 1), t_train)

In [ ]:
# 訓練データを予測
y_train = model.predict(x_train.reshape(-1, 1))
y_train

In [ ]:
# 検証データを予測
y_valid = model.predict(x_valid.reshape(-1, 1))
y_valid

In [ ]:
# テストデータを予測
y_test = model.predict(x_test.reshape(-1, 1))
y_test

In [ ]:
# 訓練データの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_train, y_train)

In [ ]:
# 検証データの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_valid, y_valid)

In [ ]:
# テストデータの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_test, y_test)

In [ ]:
plt.scatter(x_train, t_train, marker='o', label="train")
plt.scatter(x_valid, t_valid, marker='x', label="valid")
plt.plot(x_test, t_test, label="test (true)")
plt.plot(x_test, y_test, label="test (predicted)")
plt.legend()
plt.show()

K Neighbors は、近くの K 個の点の値の間を取ろうとするので、こんな予測曲線になります。

# Gradient Boosting

非深層系の機械学習で最強クラスと噂される Gradient Boosting です（厳密にはその亜種である Light GBM が最強と噂されています）。

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor()

In [ ]:
# 学習する
model.fit(x_train.reshape(-1, 1), t_train)

In [ ]:
# 訓練データを予測
y_train = model.predict(x_train.reshape(-1, 1))
y_train

In [ ]:
# 検証データを予測
y_valid = model.predict(x_valid.reshape(-1, 1))
y_valid

In [ ]:
# テストデータを予測
y_test = model.predict(x_test.reshape(-1, 1))
y_test

In [ ]:
# 訓練データの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_train, y_train)

In [ ]:
# 検証データの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_valid, y_valid)

In [ ]:
# テストデータの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_test, y_test)

In [ ]:
plt.scatter(x_train, t_train, marker='o', label="train")
plt.scatter(x_valid, t_valid, marker='x', label="valid")
plt.plot(x_test, t_test, label="test (true)")
plt.plot(x_test, y_test, label="test (predicted)")
plt.legend()
plt.show()

Gradient Boosting の「クセ」はお分かりでしょうか。最強クラスかもしれませんが、ハイパーパラメーターを適切にチューニングしないと、こんな感じに overfitting します。

# MLP

深層学習の中で最も単純なのが MLP です。

In [ ]:
from sklearn.neural_network import MLPRegressor

params = {"hidden_layer_sizes":[100]*10, "max_iter":530000}
model = MLPRegressor(**params)

In [ ]:
# 学習する
model.fit(x_train.reshape(-1, 1), t_train)

In [ ]:
# 訓練データを予測
y_train = model.predict(x_train.reshape(-1, 1))
y_train

In [ ]:
# 検証データを予測
y_valid = model.predict(x_valid.reshape(-1, 1))
y_valid

In [ ]:
# テストデータを予測
y_test = model.predict(x_test.reshape(-1, 1))
y_test

In [ ]:
# 訓練データの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_train, y_train)

In [ ]:
# 検証データの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_valid, y_valid)

In [ ]:
# テストデータの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_test, y_test)

In [ ]:
plt.scatter(x_train, t_train, marker='o', label="train")
plt.scatter(x_valid, t_valid, marker='x', label="valid")
plt.plot(x_test, t_test, label="test (true)")
plt.plot(x_test, y_test, label="test (predicted)")
plt.legend()
plt.show()

さて、色んな機械学習モデルを試すのは、以上です。

ここまでで、「検証データって、必要なの？テストデータだけ使えば十分じゃないの？」と疑問を持たれた方がいらっしゃるかもしれません。それは、次の「グリッドサーチ」以降で分かると思います。

# グリッドサーチ

訓練データには適合していても、検証データやテストデータには適合していない学習例が数多く算出されました。これが「過剰適合」(overfitting) の例です。

上記の例では、機械学習時に用いるハイパーパラメーターをわざと変な値にしたり、デフォルト値のまま使ったりしています。

実際には、ハイパーパラメーターを良い感じにチューニングして使います。その方法の一つである「グリッドサーチ」を使って、SVMのハイパーパラメーターをチューニングする例を以下に実演します。

In [ ]:
# SVMのハイパーパラメータの一つ gamma を７個用意します。
gammas = [10**n for n in range(-3, 4)]
gammas

In [ ]:
# SVMのハイパーパラメータの一つ C を７個用意します。
Cs = [10**n for n in range(-3, 4)]
Cs

用意した７個のgammaと、７個のCの全組み合わせについてSVMモデルを作成し、最も性能の良いモデルを選び出します。

In [ ]:
best_valid_score = None # 最も良いスコアを記録するための変数
best_model = None # 最も良いモデルを保存するための変数
score_record = {} # グリッドサーチの全結果を保存するための変数

for gamma in gammas: # 全ての gamma に対して
    k1 = "g={}".format(gamma)
    score_record[k1] = {}

    for C in Cs: # 全ての C に対して
        k2 = "C={}".format(C)

        # 訓練データを用いて学習
        model = SVR(kernel="rbf", C=C, gamma=gamma)
        model.fit(x_train.reshape(-1, 1), t_train)

        # 検証データを予測
        y_valid = model.predict(x_valid.reshape(-1, 1))
        mae_valid = mean_absolute_error(y_valid, t_valid)
        score_record[k1][k2] = mae_valid

        # 検証データの予測性能が最も高いモデルを保存
        if best_valid_score is None or best_valid_score > mae_valid:
            best_valid_score = mae_valid
            best_model = model

７個のgammaと７個のCに対して作成した全モデルの性能は次のようになります。

In [ ]:
import pandas as pd
pd.DataFrame(score_record).style.background_gradient(axis=None, cmap="jet")

ベストモデル賞の栄誉に輝いたのはこのモデルです。

In [ ]:
best_model

In [ ]:
# 訓練データの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_train, best_model.predict(x_train.reshape(-1, 1)))

In [ ]:
# 検証データの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_valid, best_model.predict(x_valid.reshape(-1, 1)))

In [ ]:
# テストデータの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_test, best_model.predict(x_test.reshape(-1, 1)))

In [ ]:
plt.scatter(x_train, t_train, marker='o', label="train")
plt.scatter(x_valid, t_valid, marker='x', label="valid")
plt.plot(x_test, f(x_test), label="test (true)")
plt.plot(x_test, best_model.predict(x_test.reshape(-1, 1)), label="test (SVM)")
plt.legend()
plt.show()

ここでは、「**訓練データを使って学習**」して、「**検証データの予測性能が高いモデルを選んでいる**」ことに注意してください。一方、テストデータは学習に使っていません。

# Optuna を用いたハイパーパラメーターチューニング

グリッドサーチでは、（普通は複数種類ある）ハイパーパラメーター毎に、いくつかの候補の「値」を事前に決めておいて、その候補のハイパーパラメーターの全組み合わせに対して学習を行ない、最も性能の良い組み合わせを選びます。

これに対して、Optuna というツールを使えば、ハイパーパラメーターの「値の範囲」を事前に決めておけばハイパーパラメーターチューニングができるようになります。

In [ ]:
# Optuna のインストール
!pip install optuna

次のコードで、SVM を Optuna でハイパーパラメーターチューニングするためのクラスを定義します。

In [ ]:
import copy
class best_SVR:
    def __init__(self, x_train, x_valid, t_train, t_valid):
        # 訓練データを格納
        self.x_train = x_train
        self.t_train = t_train

        # 検証データを格納
        self.x_valid = x_valid
        self.t_valid = t_valid

        # ベストモデルとスコアを格納
        self.best_score = None
        self.best_model = None

    def __call__(self, trial):
        # チューニングしたいパラメータの範囲を設定
        model_params = {}
        model_params["C"] = trial.suggest_float("C", 1e-10, 1e10, log=True)
        model_params["gamma"] = trial.suggest_float("gamma", 1e-10, 1e10, log=True)

        # SVMモデルを作成し訓練データを学習
        model = SVR(**model_params)
        model.fit(self.x_train.reshape(-1, 1), self.t_train)

        # 検証データの予測性能を評価
        score = mean_absolute_error(model.predict(x_valid.reshape(-1, 1)), self.t_valid)

        # ベストスコアが出れば、そのベストモデルを記録
        if self.best_model is None or self.best_score > score:
            self.best_score = score
            self.best_model = copy.deepcopy(model)

        # スコアを返す
        return score

次のコードで、学習を行います。

In [ ]:
import optuna

# 各種設定
optuna.logging.set_verbosity(optuna.logging.WARN)
timeout = 50
n_trials = 100
show_progress_bar = True

# 目的変数（最小化または最大化したい値）の設定
objective = best_SVR(x_train, x_valid, t_train, t_valid)

# 学習環境を立ち上げる
study = optuna.create_study(direction="minimize")

# 学習する
study.optimize(
        objective,
        timeout=timeout,
        n_trials=n_trials,
        show_progress_bar=show_progress_bar,
    )

グリッドサーチは指定した全組み合わせを満遍なく計画通りに探索するのに対して、Optunaでは、有望そうであると思われた範囲を重点的に探索し、グリッドサーチでは到達するのが難しい値に到達することができます。図示しましょう。

In [ ]:
# 性能評価指標の推移
import matplotlib.pyplot as plt

plt.plot([trial.value for trial in study.trials], label='value')
plt.grid()
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
# C の推移
import matplotlib.pyplot as plt

plt.plot([trial.params['C'] for trial in study.trials], label='C')
plt.grid()
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
# gamma の推移
import matplotlib.pyplot as plt

plt.plot([trial.params['gamma'] for trial in study.trials], label='gamma')
plt.grid()
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
# C と gamma の推移
import matplotlib.pyplot as plt

plt.plot(
    [trial.params['gamma'] for trial in study.trials], 
    [trial.params['C'] for trial in study.trials], 
    marker='o',
    alpha=0.8)
plt.grid()
plt.xscale('log')
plt.yscale('log')
plt.xlabel("Gamma")
plt.ylabel("C")
plt.show()

グリッドサーチでは到達が難しそうな場所を探索できていることがお分かりでしょうか。

ベストモデル賞の栄誉に輝いたのはこのモデルです。

In [ ]:
best_model = objective.best_model

In [ ]:
# 訓練データの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_train, best_model.predict(x_train.reshape(-1, 1)))

In [ ]:
# 検証データの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_valid, best_model.predict(x_valid.reshape(-1, 1)))

In [ ]:
# テストデータの予測性能の評価（平均絶対誤差）
mean_absolute_error(t_test, best_model.predict(x_test.reshape(-1, 1)))

In [ ]:
plt.scatter(x_train, t_train, marker='o', label="train")
plt.scatter(x_valid, t_valid, marker='x', label="valid")
plt.plot(x_test, f(x_test), label="test (true)")
plt.plot(x_test, best_model.predict(x_test.reshape(-1, 1)), label="test (SVM)")
plt.legend()
plt.show()

# 課題

下記の関数に対して、上記のコードを全て動かしてください。その結果について説明してください。

In [ ]:
f = lambda x: 0.2*np.sin(x) + 0.3*np.cos(2*x) + 0.5*np.sin(2/3*x) + 0.2*np.sin(x/3) 

<b>提出方法：</b>

下記のいずれかの方法で提出してください。

- Google Colaboratory 上で動作させたコードを ikemenmaskot@gmail.com に「共有」

- Jupyter Notebook 上で動作させたコードを ipynb 形式または html 形式にして ikemenmaskot@gmail.com に「メール送信」

# 次回

第２回は「適用範囲」というテーマでお話ししたいと思います。